# rGreat online vs local 

This notebook allows you to compare results between local and online rGreat. 

r-base == 3.6.1

r-base required for rGreat is 3.6.0

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import pandas as pd

In [3]:
rpy2.__version__

'3.5.2'

In [11]:
importr('rGREAT')

rpy2.robjects.packages.Package as a <module 'rGREAT'>

In [12]:
res_online = rpy2.robjects.r['submitGreatJob']("../../data/tests/test_data/input/07_height_snps_hg19.bed")
res_online = rpy2.robjects.r['getEnrichmentTables'](res_online)

R[write to console]: Note: On Aug 19 2019 GREAT released version 4 which supports hg38
genome and removes some ontologies such pathways. submitGreatJob()
still takes hg19 as default. hg38 can be specified by argument `species
= "hg38"`. To use the older versions such as 3.0.0, specify as
submitGreatJob(..., version = "3"). Set argument `help` to `FALSE` to
turn off this message.

R[write to console]: The default enrichment tables contain no associated genes for the input
regions. You can set `download_by = 'tsv'` to download the complete
table, but note only the top 500 regions can be retreived. See the
following link:

https://great-help.atlassian.net/wiki/spaces/GREAT/pages/655401/Export#Export-GlobalExport



In [34]:
res_local = rpy2.robjects.r['great']("../../data/tests/test_data/input/07_height_snps_hg19.bed", "MSigDB:H", "TxDb.Hsapiens.UCSC.hg19.knownGene")
res_local = rpy2.robjects.r['getEnrichmentTables'](res_local)

KeyError: "'great' not found"

In [32]:
print(res_online.names)

[1] "GO Molecular Function" "GO Biological Process" "GO Cellular Component"



In [30]:
print(res_online.rx2("GO Molecular Function").colnames)

 [1] "ID"                         "name"                      
 [3] "Binom_Genome_Fraction"      "Binom_Expected"            
 [5] "Binom_Observed_Region_Hits" "Binom_Fold_Enrichment"     
 [7] "Binom_Region_Set_Coverage"  "Binom_Raw_PValue"          
 [9] "Binom_Adjp_BH"              "Hyper_Total_Genes"         
[11] "Hyper_Expected"             "Hyper_Observed_Gene_Hits"  
[13] "Hyper_Fold_Enrichment"      "Hyper_Gene_Set_Coverage"   
[15] "Hyper_Term_Gene_Coverage"   "Hyper_Raw_PValue"          
[17] "Hyper_Adjp_BH"             



In [33]:
online = pd.DataFrame({
    "id":list(res_online.rx2("GO Molecular Function").rx2("ID"))+list(res_online.rx2("GO Biological Process").rx2("ID"))+list(res_online.rx2("GO Cellular Component").rx2("ID")),
    "term":list(res_online.rx2("GO Molecular Function").rx2("name"))+list(res_online.rx2("GO Biological Process").rx2("name"))+list(res_online.rx2("GO Cellular Component").rx2("name")),
    "binom_p_val" : list(res_online.rx2("GO Molecular Function").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Binom_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Binom_Raw_PValue")),
    "hyper_p_val":list(res_online.rx2("GO Molecular Function").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Biological Process").rx2("Hyper_Raw_PValue"))+list(res_online.rx2("GO Cellular Component").rx2("Hyper_Raw_PValue"))
    })
online.sort_values(by=["binom_p_val"])

,id,term,binom_p_val,hyper_p_val
4219,GO:2000772,regulation of cellular senescence,2.873591e-21,0.001120
4220,GO:0090342,regulation of cell aging,8.617844e-21,0.002480
4221,GO:0090402,oncogene-induced cell senescence,1.727243e-18,0.000239
4222,GO:0035986,senescence-associated heterochromatin focus as...,1.203867e-17,0.000708
17364,GO:0035985,senescence-associated heterochromatin focus,1.203867e-17,0.000708
...,...,...,...,...
9349,GO:0008090,retrograde axonal transport,1.000000e+00,1.000000
9350,GO:0008105,asymmetric protein localization,1.000000e+00,1.000000
9351,GO:0008156,negative regulation of DNA replication,1.000000e+00,1.000000
9353,GO:0008207,C21-steroid hormone metabolic process,1.000000e+00,1.000000
